In [1]:
import json, boto3, logging
from botocore.exceptions import ClientError

In [2]:
with open("./.KEYS/API_KEYS.json", "r") as f:
    key = json.load(f)

# S3 버킷 정보 get
with open("./.KEYS/DATA_SRC_INFO.json", "r") as f:
    bucket_info = json.load(f)

In [3]:
dynamo_table_name = bucket_info['restore_table_name']

In [4]:
dynamodb = boto3.resource(
        'dynamodb',
        aws_access_key_id=key['aws_access_key_id'],
        aws_secret_access_key=key['aws_secret_key'],
        region_name=key['region']
    )
table = dynamodb.Table(dynamo_table_name)

In [10]:
scan_kwargs = {}
data = []

In [11]:
# Pagination을 고려하여 모든 데이터를 가져오기
while True:
    response = table.scan(**scan_kwargs)
    data.extend(response.get('Items', []))
    scan_kwargs['ExclusiveStartKey'] = response.get('LastEvaluatedKey', None)
    if not scan_kwargs['ExclusiveStartKey']:
        break

In [19]:
data[0]

{'company_id': Decimal('3257'),
 'site_symbol': 'WAN',
 'job_title': 'Software Engineer',
 'job_prefer': '클래스101에서 사용하고 있는 기술에 익숙하시다면 빠른 적응에 도움이 됩니다. React, React Native, TypeScript, GraphQL, Apollo Node.js, Apollo Federation, MongoDB Java, Kotlin, Spring Boot, JPA, Aurora Kafka, Redis, Kubernetes',
 'crawl_url': 'https://www.wanted.co.kr/wd/105598',
 'end_date': None,
 'job_id': Decimal('105598'),
 'crawl_domain': 'www.wanted.co.kr',
 'company_name': '클래스101(Class101)',
 'get_date': Decimal('1724605651'),
 'job_tasks': '클래스101의 다양한 서비스를 설계, 개발하고 운영합니다. Discovery 클래스메이트가 온라인 클래스, 준비물, 전자책, 크리에이터, 커뮤니티 등 클래스101이 가진 모든 콘텐츠 중 원하는 것을 쉽게 발견할 수 있도록 분류, 추천 및 검색 경험을 개선합니다. Learning Experience 다양한 환경에서 영상 콘텐츠를 실시간으로 스트리밍하고, 다른 클래스메이트들 혹은 크리에이터와 직접 소통하는 커뮤니티를 만들고, 수강률과 잔존율을 높이는 제품 실험을 진행하는 등 구매 이후의 경험을 책임집니다. Creator 크리에이터가 자유롭게 콘텐츠를 만들고, 클래스메이트들과 소통하고, 수익을 극대화할 수 있도록 다양한 제품을 만듭니다. Commerce 실물 및 비실물 상품의 단건 결제와 구독 결제, 주문 처리, 재고 관리와 배송, 정산과 세금 계산까지를 아우르는 온라인 마켓플레이스 플랫폼을 구축합니다. Core 인증, 알림, 실험 등 클래